In [1]:
%%bash

wget --output-document forecasts.csv http://forecaster.rchaves.app/

--2021-05-09 22:30:58--  http://forecaster.rchaves.app/
Resolvendo forecaster.rchaves.app (forecaster.rchaves.app)... 104.21.87.72, 172.67.142.87
Conectando-se a forecaster.rchaves.app (forecaster.rchaves.app)|104.21.87.72|:80... conectado.
A requisição HTTP foi enviada, aguardando resposta... 301 Moved Permanently
Localização: https://forecaster.rchaves.app/ [redirecionando]
--2021-05-09 22:30:58--  https://forecaster.rchaves.app/
Conectando-se a forecaster.rchaves.app (forecaster.rchaves.app)|104.21.87.72|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 8245390 (7,9M) [text/csv]
Salvando em: “forecasts.csv”

     0K .......... .......... .......... .......... ..........  0% 11,7K 11m26s
    50K .......... .......... .......... .......... ..........  1% 1,51M 5m44s
   100K .......... .......... .......... .......... ..........  1% 3,28M 3m48s
   150K .......... .......... .......... .......... ..........  2% 2,75M 2m51s
   200K .......... ......

In [2]:
import pandas as pd
from yyyy_mm_dd import *
import numpy as np

In [3]:
df = pd.read_csv("forecasts.csv", parse_dates=['timestamp'])
df

,timestamp,type,forecast_delta,city,summary,precipitation,temperature,max,min
0,2021-04-26 07:40:55.852295,daily,0,Amsterdam,Partly Cloudy,1,NaN,11.0,2.0
1,2021-04-26 07:40:55.852295,daily,1,Amsterdam,Partly Cloudy,8,NaN,15.0,4.0
2,2021-04-26 07:40:55.852295,daily,2,Amsterdam,Mostly Sunny,5,NaN,16.0,6.0
3,2021-04-26 07:40:55.852295,daily,3,Amsterdam,Light Rain,82,NaN,11.0,5.0
4,2021-04-26 07:40:55.852295,daily,4,Amsterdam,Showers,47,NaN,12.0,4.0
...,...,...,...,...,...,...,...,...,...
123727,2021-05-09 20:00:41.644181,hourly,43,Cairo,Sunny,0,33.0,NaN,NaN
123728,2021-05-09 20:00:41.644181,hourly,44,Cairo,Sunny,0,32.0,NaN,NaN
123729,2021-05-09 20:00:41.644181,hourly,45,Cairo,Clear,0,29.0,NaN,NaN
123730,2021-05-09 20:00:41.644181,hourly,46,Cairo,Clear,0,28.0,NaN,NaN


In [4]:
hourly = df[df.type == 'hourly'].copy()
hourly = hourly.drop(columns=['min', 'max'])
hourly['timestamp'] = [ start_of_yyyy_mm_dd_hh(timestamp) for timestamp in hourly['timestamp'] ]
hourly['forecast_for'] = [ move_yyyy_mm_dd_hh(timestamp, delta)
                           for timestamp, delta in zip(hourly['timestamp'], hourly['forecast_delta']) ]
hourly

,timestamp,type,forecast_delta,city,summary,precipitation,temperature,forecast_for
15,2021-04-26 07:00:00,hourly,1,Amsterdam,Partly Cloudy,1,8.0,2021-04-26 08:00:00
16,2021-04-26 07:00:00,hourly,2,Amsterdam,Partly Cloudy,0,9.0,2021-04-26 09:00:00
17,2021-04-26 07:00:00,hourly,3,Amsterdam,Mostly Sunny,0,9.0,2021-04-26 10:00:00
18,2021-04-26 07:00:00,hourly,4,Amsterdam,Mostly Sunny,0,9.0,2021-04-26 11:00:00
19,2021-04-26 07:00:00,hourly,5,Amsterdam,Sunny,0,10.0,2021-04-26 12:00:00
...,...,...,...,...,...,...,...,...
123727,2021-05-09 20:00:00,hourly,43,Cairo,Sunny,0,33.0,2021-05-11 15:00:00
123728,2021-05-09 20:00:00,hourly,44,Cairo,Sunny,0,32.0,2021-05-11 16:00:00
123729,2021-05-09 20:00:00,hourly,45,Cairo,Clear,0,29.0,2021-05-11 17:00:00
123730,2021-05-09 20:00:00,hourly,46,Cairo,Clear,0,28.0,2021-05-11 18:00:00


In [40]:
to_join = hourly[hourly.forecast_delta == 0].drop(columns=['forecast_for']).rename(columns={'timestamp': 'forecast_for'}).set_index(['city', 'forecast_for'])[['precipitation']].copy()
hourly = hourly.join(
    to_join,
    on=['city', 'forecast_for'],
    how='inner',
    rsuffix="_actual"
)
hourly

,timestamp,type,forecast_delta,city,summary,precipitation,temperature,forecast_for,precipitation_actual
16,2021-04-26 07:00:00,hourly,2,Amsterdam,Partly Cloudy,0,9.0,2021-04-26 09:00:00,0
393,2021-04-26 08:00:00,hourly,1,Amsterdam,Partly Cloudy,0,9.0,2021-04-26 09:00:00,0
771,2021-04-26 09:00:00,hourly,0,Amsterdam,Mostly Sunny,0,10.0,2021-04-26 09:00:00,0
17,2021-04-26 07:00:00,hourly,3,Amsterdam,Mostly Sunny,0,9.0,2021-04-26 10:00:00,0
394,2021-04-26 08:00:00,hourly,2,Amsterdam,Partly Cloudy,0,9.0,2021-04-26 10:00:00,0
...,...,...,...,...,...,...,...,...,...
122176,2021-05-09 16:00:00,hourly,4,Cairo,Partly Cloudy,1,28.0,2021-05-09 20:00:00,2
122553,2021-05-09 17:00:00,hourly,3,Cairo,Partly Cloudy,1,29.0,2021-05-09 20:00:00,2
122930,2021-05-09 18:00:00,hourly,2,Cairo,Partly Cloudy,1,29.0,2021-05-09 20:00:00,2
123307,2021-05-09 19:00:00,hourly,1,Cairo,Clear,1,29.0,2021-05-09 20:00:00,2


In [41]:
mai9 = hourly[(hourly.city == 'Amsterdam') & (hourly.forecast_for == '2021-05-09 15:00:00')]
mai9

,timestamp,type,forecast_delta,city,summary,precipitation,temperature,forecast_for,precipitation_actual
103760,2021-05-07 16:00:00,hourly,47,Amsterdam,Isolated T-Storms,33,24.0,2021-05-09 15:00:00,5
104137,2021-05-07 17:00:00,hourly,46,Amsterdam,Isolated T-Storms,33,24.0,2021-05-09 15:00:00,5
104514,2021-05-07 18:00:00,hourly,45,Amsterdam,Isolated T-Storms,33,24.0,2021-05-09 15:00:00,5
104891,2021-05-07 19:00:00,hourly,44,Amsterdam,Scattered T-Storms,35,23.0,2021-05-09 15:00:00,5
105268,2021-05-07 20:00:00,hourly,43,Amsterdam,Isolated T-Storms,32,24.0,2021-05-09 15:00:00,5
105645,2021-05-07 21:00:00,hourly,42,Amsterdam,Mostly Cloudy,24,24.0,2021-05-09 15:00:00,5
106022,2021-05-07 22:00:00,hourly,41,Amsterdam,Mostly Cloudy,24,24.0,2021-05-09 15:00:00,5
106399,2021-05-07 23:00:00,hourly,40,Amsterdam,Mostly Cloudy,24,24.0,2021-05-09 15:00:00,5
106776,2021-05-08 00:00:00,hourly,39,Amsterdam,Mostly Cloudy,24,24.0,2021-05-09 15:00:00,5
107153,2021-05-08 01:00:00,hourly,38,Amsterdam,Mostly Cloudy,24,24.0,2021-05-09 15:00:00,5


In [42]:
from sklearn.metrics import mean_squared_error

mean_squared_error([5, 5, 5, 5, 5], [24, -24, 5, 5, 5])

240.4

In [43]:
mean_squared_error([5], [24])

361.0

In [58]:
max_hours = 36
slice_ = hourly[(hourly.city == 'Amsterdam') & (hourly.forecast_for > '2021-05-09') &
                (hourly.forecast_for < '2021-05-09 20:00:00') &
                (hourly.forecast_delta > 0) & (hourly.forecast_delta <= max_hours)].copy()

mses = {}
for i in range(1, max_hours + 1):
    delta_i_forecasts = slice_[slice_.forecast_delta == i]
    mses[i] = mean_squared_error(delta_i_forecasts['precipitation_actual'], delta_i_forecasts['precipitation'], squared=False)
mses

{1: 23.79849619309236,
 2: 26.66063528283245,
 3: 23.596386877744873,
 4: 24.179155877477783,
 5: 22.946161242993863,
 6: 23.401529414706392,
 7: 36.516758949974275,
 8: 37.14268363175204,
 9: 37.98891805167452,
 10: 38.6339092944064,
 11: 39.368702503919344,
 12: 40.4507497816139,
 13: 40.32499551342684,
 14: 40.15758432659226,
 15: 40.30084235765851,
 16: 38.156878389368444,
 17: 37.81464489349401,
 18: 38.477607979050276,
 19: 38.22509509127272,
 20: 38.499487351337415,
 21: 37.19436856073214,
 22: 38.40435830530313,
 23: 41.25019936155971,
 24: 39.628538361029406,
 25: 39.27835884662457,
 26: 39.54278162885571,
 27: 40.54756790681649,
 28: 39.21197454054592,
 29: 38.721678954464664,
 30: 38.841106819288356,
 31: 40.32499551342684,
 32: 39.58933934121261,
 33: 39.03574610128786,
 34: 34.66607624328521,
 35: 34.86741805905999,
 36: 33.36480969997865}